# Aprendizem Profunda - LSTM
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [33]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


### Input dos dados

In [34]:
df = pd.read_csv("data/big_dataset.csv")
df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")


### Função pra gravar em csv

In [35]:
def to_csv(results_df, name):
    os.makedirs("submissao2", exist_ok=True)
    path = f"submissao2/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [36]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [37]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 24276, Unique texts: 24158
Overlap between train and test after deduplication: 0


### Tokenizer

In [38]:
# Tokenize the data
tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=200)
X_val = pad_sequences(X_val_seq, maxlen=200)
X_test = pad_sequences(X_test_seq, maxlen=200)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [39]:
data_set_professor = pd.read_csv("data/dataset3_inputs.csv", sep=';', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=200)  

### Early Stopping

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=3,            
    restore_best_weights=True,           
)

### Reduce_lr

In [41]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

## LSTM

In [47]:
model_lstm = Sequential([
    Embedding(input_dim=30000, output_dim=32),  

    LSTM(16, activation="tanh", return_sequences=True, 
         kernel_regularizer=l2(0.005), recurrent_dropout=0.5), 
    Dropout(0.5),

    LSTM(8, kernel_regularizer=l2(0.005), recurrent_dropout=0.5, return_sequences=False),
    Dropout(0.5),

    BatchNormalization(),

    Dense(16, activation="relu", kernel_regularizer=l2(0.003)), 
    Dropout(0.7),  

    Dense(1, activation="sigmoid")
])

optimizer = Adam(learning_rate=0.0001, clipnorm=1.0)
model_lstm.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

model_lstm.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,  
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    shuffle=True
)   

Epoch 1/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 101s 168ms/step - accuracy: 0.5048 - loss: 1.0441 - val_accuracy: 0.6741 - val_loss: 0.9340 - learning_rate: 1.0000e-04
Epoch 2/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 69s 143ms/step - accuracy: 0.5578 - loss: 0.9105 - val_accuracy: 0.8112 - val_loss: 0.8111 - learning_rate: 1.0000e-04
Epoch 3/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 80s 165ms/step - accuracy: 0.6364 - loss: 0.7908 - val_accuracy: 0.8614 - val_loss: 0.6980 - learning_rate: 1.0000e-04
Epoch 4/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 84s 174ms/step - accuracy: 0.7105 - loss: 0.6934 - val_accuracy: 0.8857 - val_loss: 0.5857 - learning_rate: 1.0000e-04
Epoch 5/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 68s 140ms/step - accuracy: 0.7613 - loss: 0.6046 - val_accuracy: 0.9020 - val_loss: 0.4994 - learning_rate: 1.0000e-04
Epoch 6/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 67s 139ms/step - accuracy: 0.8094 - loss: 0.5297 - val_accuracy: 0.9084 - val_loss: 0.4200 - learning_rate: 1.0000e-04
Epoch 7/20
484/484 ━━━━━━━━━━━━━━━━━━━━ 68s 1

**Test Set**

In [ ]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model_lstm, X_test, test_ids, "lstm")

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Predictions saved for dnn
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Predictions saved for simple_rnn
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Predictions saved for rnn
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Predictions saved for lstm
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Predictions saved for gru
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
              precision    recall  f1-score   support

         0.0       0.52      1.00      0.68       420
         1.0       0.00      0.00      0.00       391

    accuracy                           0.52       811
   macro avg       0.26      0.50      0.34       811
weighted avg       0.27      0.52      0.35       811



c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ruben\Desktop\Minho\MEI\SI\AP\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

**Avaliação**

In [49]:
predict_and_save(model_lstm, X_professor, data_set_professor["ID"], "lstm_professor")
model_lstm.save("models/lstm.keras")  

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


## Classification Reports

In [ ]:
print(classification_report(y_test, model_lstm.predict(X_test) > 0.5))

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       420
         1.0       1.00      0.96      0.98       391

    accuracy                           0.98       811
   macro avg       0.98      0.98      0.98       811
weighted avg       0.98      0.98      0.98       811

